In [3]:
import copy
import time
import os
import glob

from ib_async import *

import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder

In [37]:
def quantReqHistoricalData(secType: str, symbol: str, endDateTime: str, durationStr: str, barSizeSetting: str):

    """
    Parameters
    ------------
    secType: str
        Type of security
        'STK' = Stock (or ETF)
        'OPT' = Option
        'FUT' = Future
        'IND' = Index
        'FOP' = Futures option
        'CASH' = Forex pair
        'CFD' = CFD
        'BAG' = Combo
        'WAR' = Warrant
        'BOND' = Bond
        'CMDTY' = Commodity
    symbol: str
        The contract (or its underlying) symbol
    endDateTime: str
        Can be set to '' to indicate the current time, or it can be given as a datetime.date or datetime.datetime, or it can be given as a string in 'yyyyMMdd HH:mm:ss' format
    durationStr: str
        How far back from the endDateTime to grab data
        Accepts changes to the number
        '60 S', '30 D', '13 W', '6 M', '10 Y'.
    barSizeSetting: str
        Must be one of: '1 secs', '5 secs', '10 secs' 15 secs', '30 secs', '1 min', '2 mins', '3 mins', '5 mins', '10 mins', '15 mins', '20 mins', '30 mins', '1 hour', '2 hours',
        '3 hours', '4 hours', '8 hours', '1 day', '1 week', '1 month'.
    """

    volume_bars, volatility_bars, all_bars = None, None, None

    try:

        contract = Contract(secType=secType, symbol=symbol, exchange='SMART', currency='USD')
        volume_bars = ib.reqHistoricalData(
            contract, endDateTime=endDateTime, durationStr=durationStr,
            barSizeSetting=barSizeSetting, whatToShow='TRADES', useRTH=True, timeout=0)

        # Convert to pandas dataframe
        volume_bars = util.df(volume_bars)
        volume_bars.sort_values(by=['date'], ascending=False, inplace=True)
        volume_bars.columns = ["date" if col == "date" else "volume_" + col for col in volume_bars.columns]

        """
        volatility_bars = ib.reqHistoricalData(
            contract, endDateTime=endDateTime, durationStr=durationStr,
            barSizeSetting=barSizeSetting, whatToShow='HISTORICAL_VOLATILITY', useRTH=True, timeout=0)

        volatility_bars = util.df(volatility_bars)
        volatility_bars.sort_values(by=['date'], ascending=False, inplace=True)
        volatility_bars.columns = ["date" if col == "date" else "volatility_" + col for col in volatility_bars.columns]
        """

        # all_bars = pd.merge(volume_bars, volatility_bars, on="date", how="inner")

    except Exception as e:

        print(e)

    finally:

        # Make sure to disconnect the client once finished with client work
        ib.disconnect()
        time.sleep(5)

    return volume_bars

def label_session(session_bars: pd.DataFrame, threshold: float = 1.0, numerical: bool = False):

    """
    Given a session_bars find out if the session is a range session or trend session 

    Parameters
    ----------
    session_bars: pd.Dataframe
        the session to investigate
    threshold: float
        threshold for deciding if trend or range
    numerical: bool
        returns 0 if trend and 1 if range

    Returns
    ----------
    label: str
        label for trend or range
    """
    
    volatility = session_bars.volume_close.pct_change().std() * (len(session_bars) ** 0.5)
    trend_score = (session_bars.volume_close.max() - session_bars.volume_close.min()) / (volatility * session_bars.volume_close.iloc[0] + 1e-6)

    label = 'trend' if abs(trend_score) > threshold else 'range'
    if numerical:
        label = 0 if abs(trend_score) > threshold else 1

    return label

def volume_bars_to_labelled_session(volume_bars: pd.DataFrame):

    """
    Convert volume_bars to labelled_sessions

    Parameters
    ----------
    volume_bars: pd.DataFrame
        dataframe of volume bars

    Returns
    ----------
    labelled_session: dict
        {date: {'session_df': session_df, 'session_label': label_session(session_df)
    """
    
    volume_bars_copy = copy.deepcopy(volume_bars)
    volume_bars_copy["date_days"] = volume_bars_copy["date"].dt.date
    volume_bars_groupby_date = {key: values for key, values in volume_bars_copy.groupby("date_days")}    
    labelled_sessions = {date: {'session_df': session_df,
                                'session_label': label_session(session_df),
                                'session_label_num': label_session(session_df, numerical=True),} for date, session_df in volume_bars_groupby_date.items()}

    return labelled_sessions



def create_bracket_order(secType: str, symbol: str, action: str, quantity: int, entry_price: float, take_profit_price: float, stop_loss_price: float):

    """
    Parameters
    ------------
    secType: str
        Type of security
        'STK' = Stock (or ETF)
        'OPT' = Option
        'FUT' = Future
        'IND' = Index
        'FOP' = Futures option
        'CASH' = Forex pair
        'CFD' = CFD
        'BAG' = Combo
        'WAR' = Warrant
        'BOND' = Bond
        'CMDTY' = Commodity
    symbol: str
        The contract (or its underlying) symbol
    action: str
        BUY or SELL
    quantity: int
        The number to buy
    entry_price: float
        Entry price, set below market price to immediately execute the order at next available time
    take_profit_price: float
        Take Profit
    stop_loss_price: float
        Stop loss

    """

    contract = Contract(secType=secType, symbol=symbol, exchange='SMART', currency='USD')
    
    # Define order parameters
    quantity = 10
    entry_price = 150  # Your entry price
    take_profit_price = 160  # Target price
    stop_loss_price = 145  # Stop loss price
    
    # Create the bracket order
    bracket = ib.bracketOrder(
        action=action,
        quantity=quantity,
        limitPrice=entry_price,
        takeProfitPrice=take_profit_price,
        stopLossPrice=stop_loss_price
    )
    
    # Place the bracket order
    for order in bracket:
        ib.placeOrder(contract, order)
    
    print("Bracket order placed successfully!")


In [69]:
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=2)
ib.reqMarketDataType(1)  # Use free, delayed, fr ozen data
volume_bars = quantReqHistoricalData(secType = "STK", symbol = "TSLA", endDateTime = "", durationStr = "10 Y", barSizeSetting = "3 mins")

volume_bars.to_csv("TSLA_10Y3mins_RTH.csv", index=False)

ib.disconnect()

In [38]:
labelled_sessions = volume_bars_to_labelled_session(volume_bars)